In [1]:
!pip install -q sktime==0.4.3

     |████████████████████████████████| 5.6MB 6.8MB/s 
     |████████████████████████████████| 22.2MB 1.5MB/s 
     |████████████████████████████████| 9.5MB 13.9MB/s 


In [2]:
!pip install -q tsfresh

     |████████████████████████████████| 92kB 6.1MB/s 
     |████████████████████████████████| 675kB 15.4MB/s 
     |████████████████████████████████| 102kB 8.2MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement cloudpickle==1.3, but you'll have cloudpickle 1.6.0 which is incompatible.
ERROR: distributed 2020.12.0 has requirement dask>=2020.12.0, but you'll have dask 2.12.0 which is incompatible.


In [74]:
import numpy as np
import pandas as pd
from sktime.utils.load_data import from_long_to_nested
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sktime.classification.compose import ColumnEnsembleClassifier, TimeSeriesForestClassifier
from sktime.utils.load_data import load_from_tsfile_to_dataframe

In [75]:
# Representing the agreed data
ts = [
    ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 1
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 2
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 3
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 4
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 5
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 6
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 7
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 8
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 9
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 10
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 11
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B'), #case 12
      ([
      (id, [10, -1, 1]), (id, [11, +1, 2]), (id, [12, -1, 3]), (id, [13, +1, 4]), (id, [14, -1, 5]), 
        (id, [15, +1, 6]), (id, [16, -1, 7]), (id, [17, +1, 8]), (id, [18, -1, 9]),(id, [19, +1, 10])
      ], 'A'), #case 13
      ([
      (id, [10, +1, 1]), (id, [11, -1, 2]), (id, [12, +1, 3]), (id, [13, -1, 4]), (id, [14, +1, 5]), 
        (id, [15, -1, 6]), (id, [16, +1, 7]), (id, [17, -1, 8]), (id, [18, +1, 9]),(id, [19, -1, 10])
      ], 'B') #case 14
]

In [66]:
# Generate long-format data from as instructed at https://www.sktime.org/en/latest/examples/loading_data.html
def generate_long_table(ts):
  num_cases= len(ts)
  series_len = len(ts[0][0])
  num_dims = len(ts[0][0][0][1])
  rows_per_case = series_len * num_dims
  total_rows = num_cases * series_len * num_dims

  case_ids = np.empty(total_rows, dtype=np.int)
  idxs = np.empty(total_rows, dtype=np.int)
  dims = np.empty(total_rows, dtype=np.int)
  vals = np.empty(total_rows, dtype=np.int)
  y = []

  for i in range(total_rows):
    case_ids[i] = int(i / rows_per_case)
    rem = i % rows_per_case
    dims[i] = int(rem / series_len)
    idxs[i] = rem % series_len
    vals[i] = ts[case_ids[i]][0][idxs[i]][1][dims[i]]

  for j in range(num_cases):
    y.append(ts[j][1])
  
  df = pd.DataFrame()
  df["case_id"] = pd.Series(case_ids)
  df["dim_id"] = pd.Series(dims)
  df["reading_id"] = pd.Series(idxs)
  df["value"] = pd.Series(vals)
  return df, np.array(y)

In [69]:
X, y = generate_long_table(ts)
X.head()


,case_id,dim_id,reading_id,value
0,0,0,0,10
1,0,0,1,11
2,0,0,2,12
3,0,0,3,13
4,0,0,4,14


In [70]:
X_nested = from_long_to_nested(X)
X_nested.head()

,dim_0,dim_1,dim_2
0,0 10 1 11 2 12 3 13 4 14 5 1...,0 -1 1 1 2 -1 3 1 4 -1 5 1 6 ...,0 1 1 2 2 3 3 4 4 5 5 ...
1,0 10 1 11 2 12 3 13 4 14 5 1...,0 1 1 -1 2 1 3 -1 4 1 5 -1 6 ...,0 1 1 2 2 3 3 4 4 5 5 ...
2,0 10 1 11 2 12 3 13 4 14 5 1...,0 -1 1 1 2 -1 3 1 4 -1 5 1 6 ...,0 1 1 2 2 3 3 4 4 5 5 ...
3,0 10 1 11 2 12 3 13 4 14 5 1...,0 1 1 -1 2 1 3 -1 4 1 5 -1 6 ...,0 1 1 2 2 3 3 4 4 5 5 ...
4,0 10 1 11 2 12 3 13 4 14 5 1...,0 -1 1 1 2 -1 3 1 4 -1 5 1 6 ...,0 1 1 2 2 3 3 4 4 5 5 ...


In [73]:
# Split the data to train/test
X_train, X_test, y_train, y_test = train_test_split(X_nested, y)
print(X.head())
# Define Classifier
classifier = ColumnEnsembleClassifier(estimators=[
        ("TSF1", TimeSeriesForestClassifier(n_estimators=100), [1]),
        ("TSF2", TimeSeriesForestClassifier(n_estimators=100), [2]),
    ])
# Fit the train data
classifier.fit(X_train, y_train)

# Predict the test data
y_pred = classifier.predict(X_test)

print("Accuracy score is: " + str(accuracy_score(y_test, y_pred)))

   case_id  dim_id  reading_id  value
0        0       0           0     10
1        0       0           1     11
2        0       0           2     12
3        0       0           3     13
4        0       0           4     14
Accuracy score is: 1.0
